### 1.Import Library

In [50]:
### 1. Import Libraries
import pandas as pd
from transformers import RobertaForQuestionAnswering, Trainer, TrainingArguments, RobertaTokenizerFast
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split


df_tonghop = pd.read_csv("../data/data_tinhhuong/csv/tonghop_data.csv")
print(df_tonghop.columns)

# Tạo cột 'context' từ 'Question' và 'Situation'
df_tonghop['context'] = df_tonghop['Question'] + " " + df_tonghop['Situation']

# Tạo Dataset từ DataFrame
dataset = Dataset.from_pandas(df_tonghop[['Question', 'context', 'Answer']])
print(dataset[0:5])

### 3. Tokenization
# Tải tokenizer từ Hugging Face
tokenizer = RobertaTokenizerFast.from_pretrained("vinai/phobert-base")

def tokenize_function(examples):
    questions = examples['Question']
    contexts = examples['context']
    answers = examples['Answer']
    
    encodings = tokenizer(
        questions,           -> inputs
        contexts,
        truncation='longest_first',
        padding='max_length',
        max_length=256,
        return_offsets_mapping=True
    )
    
    start_positions = []
    end_positions = []

    for i, (context, answer) in enumerate(zip(contexts, answers)):
        start_char = context.find(answer)
        if start_char == -1:
            # Đáp án không tìm thấy trong đoạn văn
            start_positions.append(0)
            end_positions.append(0)
            continue
        
        end_char = start_char + len(answer)
        offsets = encodings['offset_mapping'][i]
        
        start_token, end_token = None, None
        
        # Tìm vị trí start_token và end_token
        for idx, (start, end) in enumerate(offsets):
            if start <= start_char < end:
                start_token = idx
            if start < end_char <= end:
                end_token = idx
        
        # Nếu không tìm thấy start_token hoặc end_token, gán giá trị mặc định
        if start_token is None:
            start_token = 0
        if end_token is None:
            end_token = len(offsets) - 1

        start_positions.append(start_token)
        end_positions.append(end_token)
    
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return encodings


### 4. Chia dữ liệu huấn luyện và đánh giá
# Chia dữ liệu thành tập huấn luyện và tập đánh giá
train_df, eval_df = train_test_split(df_tonghop, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df[['Question', 'context', 'Answer']])
eval_dataset = Dataset.from_pandas(eval_df[['Question', 'context', 'Answer']])

# Tokenize dữ liệu huấn luyện và đánh giá
train_tokenized = train_dataset.map(tokenize_function, batched=True)
eval_tokenized = eval_dataset.map(tokenize_function, batched=True)

### 5. Tải mô hình và huấn luyện
# Tải mô hình RobertaForQuestionAnswering
model = RobertaForQuestionAnswering.from_pretrained("vinai/phobert-base")

# Định nghĩa các tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=eval_tokenized,
)

# Huấn luyện mô hình
trainer.train()


Index(['Index', 'Name', 'Time', 'Question', 'Situation', 'Answer'], dtype='object')
{'Question': ['Người dân tộc thiểu số xã nông thôn mới có được hỗ trợ BHYT?', 'Có được truy lĩnh chế độ thai sản?', 'Khi nào cần xin thêm Giấy chứng nhận nghỉ việc hưởng BHXH?', 'Đóng BHXH tự nguyện bao lâu thì được lương hưu?', 'Đóng BHXH bao lâu trước khi sinh thì được hưởng chế độ thai sản?'], 'context': ['Người dân tộc thiểu số xã nông thôn mới có được hỗ trợ BHYT? Tôi xin hỏi, một xã giai đoạn 2016-2020 là xã khu vực III, sang giai đoạn 2021-2025 là xã khu vực I, đã đạt chuẩn xã nông thôn mới thì người dân tộc thiểu số trên địa bàn xã này có thuộc nhóm đối tượng được ngân sách Nhà nước hỗ trợ đóng BHYT không?', 'Có được truy lĩnh chế độ thai sản? Tôi đóng BHXH từ tháng 6/2011 đến tháng 12/2014, tháng 8/2014 tôi sinh con. Do công ty nợ tiền BHXH nên hết thời gian nghỉ thai sản tôi chưa được thanh toán chế độ. Năm 2021 công ty đã nộp hết số tiền nợ BHXH và giải thể. Xin hỏi, bây giờ tôi làm đơn xin h

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PhobertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.














Map: 100%|██████████| 12818/12818 [00:04<00:00, 2665.40 examples/s]











Map: 100%|██████████| 10254/10254 [00:03<00:00, 3159.00 examples/s]




Map: 100%|██████████| 2564/2564 [00:00<00:00, 3255.52 examples/s]
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  2%|▏         | 485/30762 [20:02<21:00:41,  2.50s/it]

KeyboardInterrupt: 

### 2.Fine-Tuning

> data origin

In [ ]:
df_bh=pd.read_csv("../data/data_tinhhuong/csv/bhxh_bhyt_bhtn_data.csv")
df_chinhsach=pd.read_csv("../data/data_tinhhuong/csv/chinhsach_nguoicocong_data.csv")
df_gd_daotao_yte=pd.read_csv("../data/data_tinhhuong/csv/giaoduc_daotao_yte_data.csv")
df_gt_xd_tn_mt=pd.read_csv("../data/data_tinhhuong/csv/giaothong_xaydung_tainguyen_moitruong_data.csv")
df_laodong_tt=pd.read_csv("../data/data_tinhhuong/csv/laodong_tienthuong_data.csv")
df_tc_nh_dt_ct=pd.read_csv("../data/data_tinhhuong/csv/taichinh_nganhang_dautu_congthuong_data.csv")
df_linhvuckhac=pd.read_csv("../data/data_tinhhuong/csv/linhvuckhac_data.csv")

> merge data

In [ ]:
# Nối các DataFrame lại thành một DataFrame duy nhất
df_tonghop = pd.concat([df_bh, df_chinhsach, df_gd_daotao_yte, df_gt_xd_tn_mt, df_laodong_tt, df_tc_nh_dt_ct, df_linhvuckhac], ignore_index=True)

df_tonghop = df_tonghop.drop(columns='Index')

# Lưu DataFrame đã nối với việc thêm cột 'index' làm tên cột
df_tonghop.to_csv("../data/data_tinhhuong/csv/tonghop_data.csv", index_label='Index')


> Sumary

In [ ]:
df_tonghop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12818 entries, 0 to 12817
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Name       12816 non-null  object
 1   Time       12818 non-null  object
 2   Question   12818 non-null  object
 3   Situation  12818 non-null  object
 4   Answer     12818 non-null  object
dtypes: object(5)
memory usage: 500.8+ KB


In [ ]:
df_tonghop=pd.read_csv("../data/data_tinhhuong/csv/tonghop_data.csv")
df_tonghop.columns

Index(['Index', 'Name', 'Time', 'Question', 'Situation', 'Answer'], dtype='object')

> Chuẩn bị dữ liệu

In [ ]:
df_tonghop['context'] = df_tonghop['Question'] + " " + df_tonghop['Situation']

In [ ]:
dataset = Dataset.from_pandas(df_tonghop[['Question', 'context', 'Answer']])

In [ ]:
dataset[0:5]

{'Question': ['Người dân tộc thiểu số xã nông thôn mới có được hỗ trợ BHYT?',
  'Có được truy lĩnh chế độ thai sản?',
  'Khi nào cần xin thêm Giấy chứng nhận nghỉ việc hưởng BHXH?',
  'Đóng BHXH tự nguyện bao lâu thì được lương hưu?',
  'Đóng BHXH bao lâu trước khi sinh thì được hưởng chế độ thai sản?'],
 'context': ['Người dân tộc thiểu số xã nông thôn mới có được hỗ trợ BHYT? Tôi xin hỏi, một xã giai đoạn 2016-2020 là xã khu vực III, sang giai đoạn 2021-2025 là xã khu vực I, đã đạt chuẩn xã nông thôn mới thì người dân tộc thiểu số trên địa bàn xã này có thuộc nhóm đối tượng được ngân sách Nhà nước hỗ trợ đóng BHYT không?',
  'Có được truy lĩnh chế độ thai sản? Tôi đóng BHXH từ tháng 6/2011 đến tháng 12/2014, tháng 8/2014 tôi sinh con. Do công ty nợ tiền BHXH nên hết thời gian nghỉ thai sản tôi chưa được thanh toán chế độ. Năm 2021 công ty đã nộp hết số tiền nợ BHXH và giải thể. Xin hỏi, bây giờ tôi làm đơn xin hưởng chế độ thai sản của năm 2014 có được không?',
  'Khi nào cần xin thê

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("vinai/phobert-base")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PhobertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.


In [ ]:
# Định nghĩa hàm tokenize_function
def tokenize_function(examples):
    questions = examples['Question']
    contexts = examples['context']
    answers = examples['Answer']
    
    encodings = tokenizer(
        questions,
        contexts,
        truncation='longest_first',
        padding='max_length',
        max_length=256,
        return_offsets_mapping=True
    )
    
    start_positions = []
    end_positions = []
    
    for i, answer in enumerate(answers):
        context = contexts[i]
        start_char = context.find(answer)
        end_char = start_char + len(answer)

        if start_char == -1:
            start_positions.append(0)
            end_positions.append(0)
            continue

        offsets = encodings['offset_mapping'][i]
        start_token = None
        end_token = None

        for idx, (start, end) in enumerate(offsets):
            if start <= start_char < end:
                start_token = idx
            if start < end_char <= end:
                end_token = idx

        if end_token is None or end_token < start_token:
            end_token = start_token
        
        start_positions.append(start_token)
        end_positions.append(end_token)

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return encodings


In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 12818/12818 [00:08<00:00, 1486.46 examples/s]


In [ ]:
### 5. Load Model
model = RobertaForQuestionAnswering.from_pretrained("vinai/phobert-base")


Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
)

trainer.train()

  0%|          | 70/38454 [02:59<25:57:09,  2.43s/it]

IndexError: index out of range in self